In [1]:
import pandas as pd
import numpy as np
import scipy
import qutip as qt
from tqdm.notebook import tqdm
import math

In [2]:
import asyncio

In [3]:
def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)

    return wrapped

# N=0 Emitter

## Simulation

In [4]:
lamb = 3.640
fq = 5313.25
fr = 5230.2

N = 10
Nq = 3
a = qt.destroy(N)
b = qt.destroy(Nq)
kappa = 0.1
gamma = kappa
aq = 227
g = 17
r = 0
def H_drive(lamb, dfreq, damp):
    H = 0
    H += (fr + lamb - dfreq)*qt.tensor([a.dag()*a, qt.qeye(Nq)])
    H += (fq - lamb - dfreq)*qt.tensor([qt.qeye(N), b.dag()*b])
    H += g*(qt.tensor([a.dag(), b]) + qt.tensor([a, b.dag()]))
    H += -(aq/2)*qt.tensor([qt.qeye(N), b.dag()*b.dag()*b*b])
    H += damp*qt.tensor([(a + a.dag()), qt.qeye(Nq)])
    H += r*damp*qt.tensor([qt.qeye(N), b + b.dag()])
    return H

In [5]:
g**2 /((fq-fr)*(1-(fq-fr)/aq))

5.487472969053443

In [6]:
H0 = H_drive(lamb, 0, 0)
H0.eigenstates()
(H0.eigenstates()[0][2] - H0.eigenstates()[0][0])-fq, (H0.eigenstates()[0][1] - H0.eigenstates()[0][0])-fr

(np.float64(-0.0006309500076895347), np.float64(0.0006309500076895347))

In [7]:
powers = np.logspace(-3, 0.45, 201, endpoint=True)
deltas = np.linspace(-5, 5, 201, endpoint=True)
cops = [np.sqrt(kappa)*qt.tensor([a, qt.qeye(Nq)]), np.sqrt(gamma)*qt.tensor([qt.qeye(N), b])]

In [8]:
Htest = H_drive(lamb, fr, powers[-1])

In [9]:
rho = qt.steadystate(Htest, cops, method='direct')
rhoc = rho.ptrace(0)

In [10]:
np.diag(rhoc.full())

array([0.08543483+0.j, 0.16824463+0.j, 0.24363796+0.j, 0.22836527+0.j,
       0.14946434+0.j, 0.07496684+0.j, 0.03172391+0.j, 0.01248915+0.j,
       0.00452529+0.j, 0.00114777+0.j])

## Power Dependence

In [11]:
data = np.zeros([len(powers), N])

In [12]:
@background
def find_ss(power):
    H = H_drive(lamb, fr, power)
    rho = qt.steadystate(H, cops)
    rho = rho.ptrace(0)
    x = np.abs(np.diag(rho.full()))
    return x

In [ ]:
import nest_asyncio
nest_asyncio.apply()                                                       
loop = asyncio.get_event_loop()                                              
looper = asyncio.gather(*[find_ss(p) for p in powers])              
data = loop.run_until_complete(looper)                                  

In [ ]:
data = np.array(data)

In [ ]:
alphas = np.zeros(len(powers))
for i in range(len(powers)):
    alphas[i] = np.sum(data[i, :]*np.arange(N))

In [ ]:
data[:,0]

In [ ]:
data.shape

In [ ]:
ncol = []
pcol = []
dcol = []
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        ncol.append(j)
        pcol.append(powers[i])
        dcol.append(data[i,j])

In [ ]:
df_dict = {
    'n': ncol,
    'power': pcol,
    'data': dcol
}
df = pd.DataFrame(df_dict)
# df.to_csv('N0-blockade-sim.csv')

# N=1 Emitter

## Simulation

In [20]:
lamb = 2.50
fq = 5348.5
fr = 5230.2

N = 7
Nq = 3
Nb = 2
a = qt.destroy(N)
b = qt.destroy(Nq)
c = qt.destroy(Nb)
kappa = 0.1
gamma = kappa
gammab = gamma
aq = 227
g = 17
gb = 14
r = 0
def H_drive(lamb, dfreq, damp):
    H = 0
    H += (fr + lamb - dfreq)*qt.tensor([a.dag()*a, qt.qeye(Nq), qt.qeye(Nb)])
    H += (fq - lamb - dfreq)*qt.tensor([qt.qeye(N), b.dag()*b, qt.qeye(Nb)])
    H += (fr - dfreq)*qt.tensor([qt.qeye(N), qt.qeye(Nq), c.dag()*c])
    H += g*(qt.tensor([a.dag(), b, qt.qeye(Nb)]) + qt.tensor([a, b.dag(), qt.qeye(Nb)]))
    H += gb*(qt.tensor([a.dag(), qt.qeye(Nq), c]) + qt.tensor([a, qt.qeye(Nq), c.dag()]))
    H += -(aq/2)*qt.tensor([qt.qeye(N), b.dag()*b.dag()*b*b, qt.qeye(Nb)])
    H += damp*qt.tensor([(a + a.dag()), qt.qeye(Nq), qt.qeye(Nb)])
    return H

In [21]:
H0 = H_drive(lamb, 0, 0)

In [22]:
(H0.eigenstates()[0][3] - H0.eigenstates()[0][0])-fq, (H0.eigenstates()[0][2] - H0.eigenstates()[0][0])-fr, (H0.eigenstates()[0][1] - H0.eigenstates()[0][0])-fr

(np.float64(0.031205696020151663),
 np.float64(13.833860528388868),
 np.float64(-13.86506622440811))

In [23]:
powers = np.logspace(-3, 0.8, 51, endpoint=True)
deltas = np.linspace(-5, 5, 201, endpoint=True)
cops = [np.sqrt(kappa)*qt.tensor([a, qt.qeye(Nq), qt.qeye(Nb)]), np.sqrt(gamma)*qt.tensor([qt.qeye(N), b, qt.qeye(Nb)]), np.sqrt(gammab)*qt.tensor([qt.qeye(N), qt.qeye(Nq), c])]

In [24]:
fd = H0.eigenstates()[0][1] - H0.eigenstates()[0][0]
Htest = H_drive(lamb, fd, powers[-1])
fd

np.float64(5216.334933775592)

In [25]:
rho = qt.steadystate(Htest, cops, method='direct')
rhoc = rho.ptrace(0)

## Power Dependence

In [26]:
data = np.zeros([len(powers), N*Nq*Nb, N*Nq*Nb], dtype='complex')
data = []

In [27]:
@background
def find_ss(power):
    H = H_drive(lamb, fd, power)
    rho = qt.steadystate(H, cops)
    x = rho
    return x

In [ ]:
import nest_asyncio
nest_asyncio.apply()                                                       
loop = asyncio.get_event_loop()                                              
looper = asyncio.gather(*[find_ss(p) for p in powers])              
data = loop.run_until_complete(looper)                                  

In [29]:
psim = (1/np.sqrt(2))*(qt.tensor([qt.basis(N, 0), qt.basis(Nb, 1)]) - qt.tensor([qt.basis(N, 1), qt.basis(Nb, 0)]))
psip = (1/np.sqrt(2))*(qt.tensor([qt.basis(N, 0), qt.basis(Nb, 1)]) + qt.tensor([qt.basis(N, 1), qt.basis(Nb, 0)]))
vacs = qt.tensor([qt.basis(N, 0), qt.basis(Nb, 0)])

In [30]:
vac = np.zeros(len(powers))
sp = np.zeros(len(powers))
sm = np.zeros(len(powers))
for i, p in enumerate(tqdm(powers)):
    rho = data[i].ptrace([0,2])
    vac[i] = np.abs(rho.overlap(vacs))
    sp[i] = np.abs(rho.overlap(psip))
    sm[i] = np.abs(rho.overlap(psim))

  0%|          | 0/51 [00:00<?, ?it/s]

In [ ]:
data.shape